Generate 1 random point for each 100 jobs in every block, and apply HDBSCAN to identify clusters in each city. Then select largest/most central cluster in each city to be downtown. Do this for US and Canada.

### Install/load packages

In [ ]:
# pip install folium matplotlib mapclassify

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib
import mapclassify
import numpy as np
import shapely
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import hdbscan

### Combine US and Canada data

In [ ]:
data_path = "C:/Users/jpg23/data/downtownrecovery/lehd_new_downtowns/"

In [ ]:
dc_wichita = gpd.read_file(data_path + "cities_lehd_jobs_block_DC_Wichita.geojson") # Washington DC + Wichita

In [ ]:
# canada = gpd.read_file(data_path + "canada_DA_jobs.geojson") # Canada
# us_b = gpd.read_file(data_path + "cities_lehd_jobs_block.geojson") # US - block

In [ ]:
# canada.head()

In [ ]:
# canada_sf = gpd.read_file(data_path + "reprojected_canada_DA.geojson")[['DAUID', 'geometry']]

In [ ]:
# canada_sf.head()

In [ ]:
# canada.id.nunique()

In [ ]:
# canada_sf.DAUID.nunique()

In [ ]:
# canada_newgeom = canada_sf.merge(canada.drop('geometry', axis=1), left_on='DAUID', right_on='id', how='inner').drop(columns = ['DAUID'])

In [ ]:
# canada_newgeom.head()

In [ ]:
# canada_newgeom.id.nunique()

In [ ]:
# us_b.head()

In [ ]:
# canada_newgeom.crs

In [ ]:
# us_b.crs

In [ ]:
dc_wichita_reproj = dc_wichita.to_crs('EPSG:4326')

In [ ]:
dc_wichita_reproj.head()

In [ ]:
# # reproject so they're in the same CRS
# us_b_reproj = us_b.to_crs('EPSG:4326')

In [ ]:
# us_b_reproj.crs

In [ ]:
# us_b_reproj.crs == canada_newgeom.crs

In [ ]:
# # gut check - are jobs_per_sq_meter relatievly similar for US & Canada?
# print(canada_newgeom.jobs_per_sq_meter.mean())
# us_b_reproj.jobs_per_sq_meter.mean()

In [ ]:
# print(canada_newgeom.jobs_per_sq_meter.median())
# us_b_reproj.jobs_per_sq_meter.median()

In [ ]:
# # stack the datasets
# canada_final = canada_newgeom.rename(columns={"CMANAME": "place"})[['id', 'total_jobs', 'jobs_per_sq_meter', 'place', 'geometry']]

# us_b_final = us_b_reproj.rename(columns={"block":"id", "city":"place"})[['id', 'total_jobs', 'jobs_per_sq_meter', 'place', 'geometry']]

In [ ]:
# df = pd.concat([us_b_final, canada_final])

In [ ]:
# df.head()

In [ ]:
# df.place.unique()

In [ ]:
# df.crs

In [ ]:
# # Drop the Quebec part of Ottawa
# df = df[df['place']!='Ottawa - Gatineau (partie du Québec / Quebec part)']

In [ ]:
# df.place.unique()

In [ ]:
# type(df)

In [ ]:
# df.plot()

### In each block/dissemination area, randomly scatter 1 point for every 100 jobs

In [ ]:
# scatter random points in polygon

def Random_Points_in_Bounds(polygon, number):   
    minx, miny, maxx, maxy = polygon.bounds
    x = np.random.uniform(minx, maxx, number*4)
    y = np.random.uniform(miny, maxy, number*4)
    gdf_poly = gpd.GeoDataFrame(index=["myPoly"], geometry=[polygon])
    df = pd.DataFrame()
    df['points'] = list(zip(x,y))
    df['points'] = df['points'].apply(shapely.geometry.Point)
    gdf_points = gpd.GeoDataFrame(df, geometry='points')
    Sjoin = gpd.sjoin(gdf_points, gdf_poly, op="within", how='left')
    pnts_in_poly = gdf_points[Sjoin.index_right=='myPoly']
    return pnts_in_poly['points'].tolist()[0:number]

In [ ]:
dc_wichita_reproj['jobs_hundreds'] = round((dc_wichita_reproj['total_jobs']/100), 0)
dc_wichita_reproj_100 = dc_wichita_reproj[dc_wichita_reproj['jobs_hundreds']>0].copy()
dc_wichita_reproj_100['points'] = dc_wichita_reproj_100[['geometry', 'jobs_hundreds']].apply(lambda x: Random_Points_in_Bounds(x[0], int(x[1])), axis=1)
dc_wichita_reproj_100_points = dc_wichita_reproj_100.explode('points') 

In [ ]:
dc_wichita_reproj_100_points.head()

In [ ]:
dc_wichita_dbscan = gpd.GeoDataFrame(dc_wichita_reproj_100_points[['points', 'jobs_per_sq_meter', 'city']].copy(), geometry='points')
dc_wichita_dbscan['point_lon'] = dc_wichita_dbscan['points'].x
dc_wichita_dbscan['point_lat'] = dc_wichita_dbscan['points'].y

dc_wichita_dbscan.to_file(data_path + 'block_random_pts_DC_wichita.geojson', driver='GeoJSON') 

In [ ]:
# # divide # of total jobs by 100
# df['jobs_hundreds'] = round((df['total_jobs']/100), 0)

# # filter to only blocks / dissemination areas where there are at least 100 jobs
# df_100 = df[df['jobs_hundreds']>0].copy()

# # create randomly scattered points in each block / dissemination area
# df_100['points'] = df_100[['geometry', 'jobs_hundreds']].apply(lambda x: Random_Points_in_Bounds(x[0], int(x[1])), axis=1)

In [ ]:
# df_100.head()

In [ ]:
# # create a row for each set of points
# df_100_points = df_100.explode('points')

In [ ]:
# df_100_points.head()

In [ ]:
# # initialize format for dbscan
# df_dbscan = gpd.GeoDataFrame(df_100_points[['id', 'points', 'jobs_per_sq_meter', 'place']].copy(), geometry='points')
# df_dbscan['point_lon'] = df_dbscan['points'].x
# df_dbscan['point_lat'] = df_dbscan['points'].y

In [ ]:
# df_dbscan.head()

In [ ]:
# df_dbscan.to_file(data_path + 'block_random_pts.geojson', driver='GeoJSON')  

In [ ]:
df_dbscan = gpd.read_file(data_path + "block_random_pts.geojson")

### Prepare data for HDBSCAN

See [hdbscan documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.HDBSCAN.html) and [documentation on parameter selection](https://hdbscan.readthedocs.io/en/latest/parameter_selection.html).

In [ ]:
df_dbscan_notnull = df_dbscan[df_dbscan['point_lon'].notna()]

In [ ]:
df_dbscan_notnull.head()

In [ ]:
# what % of rows were not NA?
df_dbscan_notnull.shape[0]/df_dbscan.shape[0]

In [ ]:
# standardize lat/long values
X = StandardScaler().fit_transform(df_dbscan_notnull[['point_lon', 'point_lat']].copy())

In [ ]:
type(X)

In [ ]:
X

### Explore points pre-HDBSCAN in specific cities

In [ ]:
# create map of all points in city
def explore_pts(city_name):
    pts_only_city = df_dbscan_notnull[df_dbscan_notnull['place']==city_name].set_crs(4326)

    return(pts_only_city.explore(
        tiles="CartoDB positron",
        style_kwds=dict(opacity=.5, fillOpacity=.5)
    ))

In [ ]:
explore_pts('San Francisco CA')

In [ ]:
explore_pts('New York NY')

In [ ]:
explore_pts('Chicago IL')

In [ ]:
explore_pts('Albuquerque NM')

In [ ]:
explore_pts('Vancouver')

In [ ]:
explore_pts('Calgary')

In [ ]:
explore_pts('Toronto')

In [ ]:
explore_pts('Philadelphia PA')

In [ ]:
explore_pts('Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_pts('Edmonton')

In [ ]:
explore_pts('Montréal')

### Apply HDBSCAN and explore clusters

See [this link](https://hdbscan.readthedocs.io/en/latest/parameter_selection.html) and [this Stack Overflow post](https://stackoverflow.com/questions/67898039/hdbscan-difference-between-parameters) for a helpful explanation of how the parameters work.

In [ ]:
def create_clusters_tuned(which_min_cluster_size, which_min_samples, which_cluster_selection_epsilon):
    
    clusterer_tuned = hdbscan.HDBSCAN(min_cluster_size = which_min_cluster_size,
                                      min_samples = which_min_samples,
                                      cluster_selection_epsilon = which_cluster_selection_epsilon,
                                      core_dist_n_jobs=1)
    
    # append results to original database
    df_dbscan_notnull_tuned = df_dbscan_notnull
    
    clusterer_tuned.fit(X)
    
    df_dbscan_notnull_tuned['cluster'] = clusterer_tuned.labels_
    df_dbscan_notnull_tuned.head()

    # Filter out non-clustered points and set CRS
    clusters_only_tuned = df_dbscan_notnull_tuned[df_dbscan_notnull_tuned["cluster"]!=-1].set_crs(4326)
    clusters_only_tuned['cluster'] = clusters_only_tuned['cluster'].astype(str)
    
    return(clusters_only_tuned)

In [ ]:
# create interactive map of clusters
# see https://matplotlib.org/stable/users/explain/colors/colormaps.html for cmap options

def explore_clusters_tuned(tuned_df, city_name):
    
    df = tuned_df[tuned_df['place']==city_name]
    
    return(df.explore(
        column="cluster",
        tiles="CartoDB positron",
        style_kwds=dict(opacity=.5, fillOpacity=.5),
        cmap='Spectral'
    ))

#### min_cluster_size = 25, min_samples = 25, cluster_selection_epsilon = 0

In [ ]:
df_25_25_0 = create_clusters_tuned(which_min_cluster_size = 25, 
                                   which_min_samples = 25, 
                                   which_cluster_selection_epsilon = 0)

In [ ]:
explore_clusters_tuned(df_25_25_0, 'San Francisco CA')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'New York NY')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Chicago IL')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Albuquerque NM')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Vancouver')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Calgary')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Toronto')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Philadelphia PA')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Edmonton')

In [ ]:
explore_clusters_tuned(df_25_25_0, 'Montréal')

#### min_cluster_size = 100, min_samples = 25, cluster_selection_epsilon = 0

In [ ]:
df_100_25_0 = create_clusters_tuned(which_min_cluster_size = 100, 
                                    which_min_samples = 25, 
                                    which_cluster_selection_epsilon = 0)

In [ ]:
explore_clusters_tuned(df_100_25_0, 'San Francisco CA')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Philadelphia PA')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Toronto')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'New York NY')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Calgary')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Albuquerque NM')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Montréal')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Vancouver')

In [ ]:
explore_clusters_tuned(df_100_25_0, 'Edmonton')

#### min_cluster_size = 100, min_samples = 100, cluster_selection_epsilon = 0

In [ ]:
df_100_100_0 = create_clusters_tuned(which_min_cluster_size = 100, 
                                     which_min_samples = 100, 
                                     which_cluster_selection_epsilon = 0)

In [ ]:
explore_clusters_tuned(df_100_100_0, 'San Francisco CA')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'New York NY')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Chicago IL')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Albuquerque NM')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Vancouver')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Calgary')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Toronto')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Philadelphia PA')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Edmonton')

In [ ]:
explore_clusters_tuned(df_100_100_0, 'Montréal')

#### min_cluster_size = 100, min_samples = 100, cluster_selection_epsilon = .00005

In [ ]:
df_100_100_5 = create_clusters_tuned(which_min_cluster_size = 100, 
                                     which_min_samples = 100, 
                                     which_cluster_selection_epsilon = .00005)

In [ ]:
explore_clusters_tuned(df_100_100_5, 'San Francisco CA')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'New York NY')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Chicago IL')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Albuquerque NM')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Vancouver')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Calgary')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Toronto')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Philadelphia PA')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Edmonton')

In [ ]:
explore_clusters_tuned(df_100_100_5, 'Montréal')

#### min_cluster_size = 50, min_samples = 50, cluster_selection_epsilon = .00005

In [ ]:
df_50_50_5 = create_clusters_tuned(which_min_cluster_size = 50, 
                                   which_min_samples = 50, 
                                   which_cluster_selection_epsilon = .00005)

In [ ]:
explore_clusters_tuned(df_50_50_5, 'San Francisco CA')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'New York NY')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Chicago IL')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Albuquerque NM')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Vancouver')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Calgary')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Toronto')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Philadelphia PA')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Nashville-Davidson metropolitan government (balance) TN')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Edmonton')

In [ ]:
explore_clusters_tuned(df_50_50_5, 'Montréal')

### Create concave hull

In [ ]:
def create_hull(df, city_name, cluster_name, which_ratio):
    
    one_cluster = df[(df['place']==city_name) & (df['cluster']==cluster_name)][['geometry']]
    
    cluster_union = one_cluster.unary_union
    
    hull = shapely.concave_hull(cluster_union, ratio = which_ratio)
    hull_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[hull])
    
    m = one_cluster.explore(
        tiles="CartoDB positron",
        color = "red",
        style_kwds=dict(opacity=.5, fillOpacity=.5)
    )

    m = hull_gdf.explore(
            m = m,
            tiles="CartoDB positron",
            style_kwds=dict(opacity=.5, fillOpacity=.5)
        )

    folium.LayerControl().add_to(m)

    return(m)    

In [ ]:
create_hull(df_100_100_0, 'Edmonton', '322', 0.7)

In [ ]:
create_hull(df_100_100_5, 'San Francisco CA', '285', 0.7)

In [ ]:
create_hull(df_100_25_0, 'Philadelphia PA', '438', 0.7)